## Interpolation

Essayer H1700010. Puisque les données de météo SYNOP sont à chaque 3h, et les donnée de débit sont à chaque 2h, on voudrait interpoler les deux données de sortes qu'elles sont tous à chaque 1h. On utilise d'abord l'interpolation linéaire.



In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
import matplotlib.pyplot as plt

In [ ]:
for i in range(10,21):
    data = pd.read_csv("../Data/Extraction_Hydro/Seine/H1700010_20"+ str(i) +".csv", encoding="utf-8", sep=";")
    data["Date"]= data["Date"].str.split(pat="+",expand = True)

    meteo["Date"]= pd.to_datetime(meteo["Date"])
    meteo = meteo.sort_values(by="Date")
    meteo= meteo.drop_duplicates("Date")
    meteo = meteo.set_index('Date').resample('1H')
    interpolated = meteo.interpolate(method='linear')
    interpolated.to_csv(str(i)+".csv", header = True)